In [6]:
from vectordb_module import FAISSDatabaseManager
import pandas as pd
train_df = pd.read_csv('train.csv')
# DB 생성 및 검색 예시
train_db_manager = FAISSDatabaseManager(
    db_path='./knn_train',  # DB를 저장할 경로
    chunk_strategy='recursive',  # 텍스트를 청크로 나누는 전략
    search_strategy='knn_best_field',  # 검색 전략
    embedding_model_name="intfloat/multilingual-e5-large"
)
train_db_manager.make_db(train_df)
print("DB 생성 완료")

test_df = pd.read_csv('test.csv')
test_db_manager = FAISSDatabaseManager(
    db_path='./knn_test',  # DB를 저장할 경로
    chunk_strategy='recursive',  # 텍스트를 청크로 나누는 전략
    search_strategy='knn_best_field',  # 검색 전략
    embedding_model_name="intfloat/multilingual-e5-large"
)
test_db_manager.make_db(train_df)
print("DB 생성 완료")

# Fewshot DB 생성
fewshot_manager = FAISSDatabaseManager(db_path='./knn_fewshot',
                                       chunk_strategy='paragraph',
                                       search_strategy="knn_best_field",  # 검색 전략
                                       embedding_model_name="intfloat/multilingual-e5-large")
fewshot_manager.make_db(train_df,fewshot=True)
print("DB 생성 완료")

In [ ]:
print('Test 1')
query = train_df.iloc[309]['Question']
print('Query:', query)
# 쿼리를 기반으로 검색 수행
results = test_db_manager.search_documents(query, top_k=5)
# 결과 출력
for i, result in enumerate(results):
    print(f"Result {i+1}: {result.metadata}...")
    print(f"Result {i+1}: {result.page_content}")

Test 1
Query: 외국인 배우자 수급권 부여는 언제부터 시행되었는가?
Result 1: {'source': './train_source/재정통계해설.pdf', 'page': 50}...
Result 1: 2018 2019
01. 통합재정수지 15.1조원 흑자 6.5조원 흑자
02. 관리재정수지 31.4조원 적자 37.6조원 적자
Result 2: {'source': './train_source/재정통계해설.pdf', 'page': 107}...
Result 2: II. 재정통계
재정지출
Result 3: {'source': './train_source/2024 나라살림 예산개요.pdf', 'page': 306}...
Result 3: | 별첨 |
작성자 및
편집자 명단
Result 4: {'source': './train_source/재정통계해설.pdf', 'page': 77}...
Result 4: 재정수입II. 재정통계
Result 5: {'source': './train_source/재정통계해설.pdf', 'page': 49}...
Result 5: 재정수지II. 재정통계


In [ ]:
print('Test 1')
query =  test_df.iloc[26]['Question']
print('Query:', query)
# 쿼리를 기반으로 검색 수행
results = test_db_manager.search_documents(query, top_k=3)
# 결과 출력
for i, result in enumerate(results):
    print(f"Result {i+1}: {result.metadata}...")
    print(f"Result {i+1}: {result.page_content}")
    

Test 1
Query: 하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?
Result 1: {'source': './train_source/2024 나라살림 예산개요.pdf', 'page': 4}...
Result 1: 제1장
2024년도 예산안 및  
기금운용계획안
대통령 시정연설
제2장
2024년도 예산안 및  
기금운용계획안 등에 대한
기획재정부 장관의 제안설명제 1 부
2024년도  
예산안 시정연설 
및 제안설명
Result 2: {'source': './train_source/2024 나라살림 예산개요.pdf', 'page': 26}...
Result 2: 제 2  장
2024년도 예산안 및  
기금운용계획안 등에 대한
기획재정부 장관의
제안설명
Result 3: {'source': './train_source/월간 나라재정 2023년 12월호.pdf', 'page': 65}...
Result 3: 재정정보화를 통해
국가 재정 업무 발전에 
기여합니다
데이터 기반 재정 혁신을 선도하는 
재정 전문기관이 되겠습니다.


In [ ]:
query = test_df.iloc[26]['Question']
results = fewshot_manager.search_documents(query, top_k=3)

print('Test 2')

print('Query:', query)
# 결과 출력
for i, result in enumerate(results):
    if isinstance(result, dict):  # Few-shot 방식의 결과
        print(f"Result {i+1}: {result}...")
    else:  # 일반 문서 검색 결과
        print(f"Result {i+1}: {result.page_content[:200]}...")

Test 2
Query: 하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?
Result 1: {'Source': '2024년도 성과계획서(총괄편)', 'Source_path': './train_source/2024년도 성과계획서(총괄편).pdf', 'Question': '2024년에 예산이 증가하는 부처와 감소하는 부처의 차이는 무엇인가?', 'Answer': '2024년에는 국세청과 관세청 등의 부처에서 예산이 증가하는 반면, 외교부와 통일부 등에서는 예산이 감소할 것으로 예상된다.'}...
Result 2: {'Source': '2024년도 성과계획서(총괄편)', 'Source_path': './train_source/2024년도 성과계획서(총괄편).pdf', 'Question': '국가재정법에 따라 성과계획서를 작성해야 하는 대상기관은 누구인가요?', 'Answer': '국가재정법에 따라 성과계획서를 작성해야 하는 대상기관은 국가재정법(제4장의2)에 따라 예산요구서를 제출하는 모든 중앙관서의 장과 기금관리주체입니다.'}...
Result 3: {'Source': '2024년도 성과계획서(총괄편)', 'Source_path': './train_source/2024년도 성과계획서(총괄편).pdf', 'Question': '국가재정법에 따라 성과계획서를 작성해야 하는 대상기관은 누구인가요?', 'Answer': '국가재정법에 따라 성과계획서를 작성해야 하는 대상기관은 국가재정법(제4장의2)에 따라 예산요구서를 제출하는 모든 중앙관서의 장과 기금관리주체입니다.'}...
